In [1]:
!pip install psycopg2 -q
!pip install -U sentence-transformers -q


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import psycopg2
from sentence_transformers import SentenceTransformer

c:\Users\Dinesh-AIZENIT\Desktop\testing\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Postgres connection

In [3]:
import yaml
with open("config.yaml") as yaml_file:
    config = yaml.safe_load(yaml_file)
print(config.keys())

dict_keys(['database', 'user', 'host', 'password', 'port'])


In [4]:
connection = psycopg2.connect(
    database = config['database'], 
    user = config['user'],
    host= config['host'],
    password = config['password'],
    port = config['port']
)

In [5]:
connection.autocommit = True

In [6]:
cur = connection.cursor()

In [93]:
cur.execute("CREATE EXTENSION vector;")

## Load embedding model

In [7]:
from sentence_transformers import SentenceTransformer
embedding_model_name = 'all-MiniLM-L6-v2'
embedding_length = 384
model = SentenceTransformer(embedding_model_name)

## Get all collections

In [8]:
# retrive all collections
cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_type = 'BASE TABLE';")
cur.fetchall()

[('dev_100',), ('dev_99',)]

## Create collection

In [9]:
# Create a collection
collection_name = 'collection1'
cur.execute("CREATE TABLE IF NOT EXISTS {} (id SERIAL PRIMARY KEY,text TEXT,embedding vector({}))".format(collection_name, embedding_length))

In [10]:
# retrive all collections
cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_type = 'BASE TABLE';")
cur.fetchall()

[('dev_100',), ('dev_99',), ('collection1',)]

## Insert Data

In [11]:
# perform embedding
texts = ["Language Models are designed to understand and generate human-like text based on patterns learned from vast amounts of data", 
         "Blockchain is a decentralized, distributed ledger technology that records transactions across a network of computers",
         "Unsupervised learning algorithms explore and extract patterns from unlabeled data, discovering inherent structures."]
embeddings = model.encode(texts)

In [12]:
# insert data to a collection
collection_name = 'collection1'
for text, embedding in zip(texts, embeddings):
    text = f"'{text}'"
    embedding = f"'{list(embedding)}'"
    query = "INSERT INTO {} (text, embedding) VALUES ({}, {})"
    cur.execute(query.format(collection_name, text, embedding))

In [13]:
# Retrieve data from a collection
collection_name = 'collection1'
cur.execute("SELECT * from {};".format(collection_name))
rows = cur.fetchall()
for row in rows:
    print(row)

(1, 'Language Models are designed to understand and generate human-like text based on patterns learned from vast amounts of data', '[-0.047761235,-0.019641073,0.01520515,0.02381487,-0.018090766,-0.011360079,-0.012329591,-0.06597559,0.0829352,0.018626727,0.035254095,0.0058100414,0.104082525,0.051070407,0.05369032,0.014387948,0.018244317,0.0042129406,-0.05614495,-0.087473266,0.06230104,0.054781977,0.021169728,-0.010448991,-0.0008699052,0.07676006,0.01927369,-0.018349482,0.08885334,-0.01368344,0.058557298,0.055071745,0.06572143,0.0694621,-0.01110623,0.068293914,-0.06394991,0.042980507,0.018547412,0.005229714,-0.014602962,-0.029589104,0.050016455,0.03642921,0.14434785,0.005375769,-0.09471201,0.017759005,0.018327951,0.045767624,-0.13617975,-0.027419593,0.016515851,-0.0124821905,0.0040220143,-0.016820919,0.011717512,0.009873014,-0.0053012804,-0.06390686,-0.06567463,-0.089373685,-0.052117616,-0.018808445,-0.003700797,-0.01587948,-0.04254123,0.037892118,0.048663877,-0.02345827,-0.07856017,0.09

## Update Data

In [14]:
id = 1
text = "Evaluating the quality of text generated by LLMs involves assessing factors like coherence, relevance, grammaticality, and avoidance of biases"
embedding = model.encode(text)

In [15]:
text = f"'{text}'"
embedding = f"'{list(embedding)}'"
query = "UPDATE {} SET text = {}, embedding = {} WHERE id = {};"
cur.execute(query.format(collection_name, text, embedding, id))

In [16]:
# Retrieve data from a collection
collection_name = 'collection1'
cur.execute("SELECT * from {};".format(collection_name))
rows = cur.fetchall()
for row in rows:
    print(row)

(2, 'Blockchain is a decentralized, distributed ledger technology that records transactions across a network of computers', '[-0.020532077,0.067246586,-0.06666563,-0.017352745,-0.05060933,-0.036585327,-0.007787535,0.06466071,0.0347426,-0.010263423,-0.033427335,0.048607472,0.01849847,0.012785908,-0.03280343,-0.014673569,-0.007084569,-0.0043049683,0.037324958,-0.033627328,-0.025599457,0.011012753,-0.018487321,0.078501716,-0.0702774,0.09417117,0.002306732,-0.069172606,-0.08906005,-0.0004151572,0.013418324,0.025472412,-0.0051868027,0.08916495,-0.11387443,0.02763477,0.06558166,0.019820295,0.03555495,-0.045001894,0.07480875,-0.006841119,0.043053642,-0.04621728,0.008158095,0.03373843,-0.04181336,0.013803797,-0.052803066,0.03345758,-0.052283496,-0.019925758,0.036677405,0.11220801,0.026911132,0.0066968785,0.034910645,-0.08409673,0.003986545,-0.11331436,0.05118383,0.00866413,0.026235059,0.0445553,0.07438228,0.02829372,0.012591799,0.107294485,-0.022719236,-0.030074894,0.082078315,-0.0214938,-0.02

## Indexing

In [17]:
# create indexing on cosine distance
collection_name = 'collection1'
cur.execute("CREATE INDEX ON {} USING ivfflat (embedding vector_cosine_ops);".format(collection_name))

## Query

In [18]:
collection_name = 'collection1'
query = "SELECT * FROM {} ORDER BY embedding <=> {} LIMIT 1;"

In [19]:
# querying data
new_text = 'Fine-tuning a Language Model involves tailoring its learned knowledge to a specific domain or task, enhancing its performance in specialized applications'
new_embedding = f"'{list(model.encode(new_text))}'"
cur.execute(query.format(collection_name, new_embedding))
result = cur.fetchall()
print(result)

[(1, 'Evaluating the quality of text generated by LLMs involves assessing factors like coherence, relevance, grammaticality, and avoidance of biases', '[0.078616,-0.012130707,-0.019362476,-0.0024883754,0.029870493,0.0050918134,-0.012387832,-0.021117948,0.08590975,0.020198861,-0.0057833362,0.035163805,0.07646394,-0.04974109,0.018758753,0.047687855,0.09104919,-0.029404376,-0.034916736,-0.07462027,0.09172236,0.10966879,0.039368317,-0.005547939,0.006228844,-0.0035988912,-0.052739523,0.031446293,0.05194785,-0.011763605,-0.060656667,0.11449138,0.023811959,0.009315515,-0.0050136466,0.05745061,-0.002767515,0.02716924,0.016022235,0.017980516,-0.0060627647,-0.013615502,0.027240409,-0.009423546,0.076952316,-0.0035158624,-0.062408715,0.0225243,-0.17634772,0.062070098,-0.06763398,-0.04185271,-0.0628717,-0.03126169,-0.031771876,0.06794475,-0.058902852,0.07441166,-0.021198498,-0.061384775,-0.039396252,-0.07838002,-0.080589354,0.06765076,0.10341633,-0.03320427,0.003177425,0.03804356,-0.069155276,0.011

In [20]:
# querying data
new_text = 'Private and public blockchains serve different purposes, with public blockchains open to anyone and private blockchains restricted to authorized participants'
new_embedding = f"'{list(model.encode(new_text))}'"
cur.execute(query.format(collection_name, new_embedding))
result = cur.fetchall()
print(result)

[(2, 'Blockchain is a decentralized, distributed ledger technology that records transactions across a network of computers', '[-0.020532077,0.067246586,-0.06666563,-0.017352745,-0.05060933,-0.036585327,-0.007787535,0.06466071,0.0347426,-0.010263423,-0.033427335,0.048607472,0.01849847,0.012785908,-0.03280343,-0.014673569,-0.007084569,-0.0043049683,0.037324958,-0.033627328,-0.025599457,0.011012753,-0.018487321,0.078501716,-0.0702774,0.09417117,0.002306732,-0.069172606,-0.08906005,-0.0004151572,0.013418324,0.025472412,-0.0051868027,0.08916495,-0.11387443,0.02763477,0.06558166,0.019820295,0.03555495,-0.045001894,0.07480875,-0.006841119,0.043053642,-0.04621728,0.008158095,0.03373843,-0.04181336,0.013803797,-0.052803066,0.03345758,-0.052283496,-0.019925758,0.036677405,0.11220801,0.026911132,0.0066968785,0.034910645,-0.08409673,0.003986545,-0.11331436,0.05118383,0.00866413,0.026235059,0.0445553,0.07438228,0.02829372,0.012591799,0.107294485,-0.022719236,-0.030074894,0.082078315,-0.0214938,-0.0

## Distances

In [21]:
# Retrieve data from a collection
collection_name = 'collection1'
cur.execute("SELECT * from {};".format(collection_name))
rows = cur.fetchall()
for row in rows:
    print(row)

(2, 'Blockchain is a decentralized, distributed ledger technology that records transactions across a network of computers', '[-0.020532077,0.067246586,-0.06666563,-0.017352745,-0.05060933,-0.036585327,-0.007787535,0.06466071,0.0347426,-0.010263423,-0.033427335,0.048607472,0.01849847,0.012785908,-0.03280343,-0.014673569,-0.007084569,-0.0043049683,0.037324958,-0.033627328,-0.025599457,0.011012753,-0.018487321,0.078501716,-0.0702774,0.09417117,0.002306732,-0.069172606,-0.08906005,-0.0004151572,0.013418324,0.025472412,-0.0051868027,0.08916495,-0.11387443,0.02763477,0.06558166,0.019820295,0.03555495,-0.045001894,0.07480875,-0.006841119,0.043053642,-0.04621728,0.008158095,0.03373843,-0.04181336,0.013803797,-0.052803066,0.03345758,-0.052283496,-0.019925758,0.036677405,0.11220801,0.026911132,0.0066968785,0.034910645,-0.08409673,0.003986545,-0.11331436,0.05118383,0.00866413,0.026235059,0.0445553,0.07438228,0.02829372,0.012591799,0.107294485,-0.022719236,-0.030074894,0.082078315,-0.0214938,-0.02

In [22]:
collection_name = 'collection1'
new_text = 'Fine-tuning a Language Model involves tailoring its learned knowledge to a specific domain or task, enhancing its performance in specialized applications'
new_embedding = f"'{list(model.encode(new_text))}'"
query = "SELECT 1 - (embedding <=> {}) AS cosine_similarity FROM {};"
cur.execute(query.format(new_embedding, collection_name))
result = cur.fetchall()
print(result)

[(-0.002660304307937622,), (0.1958264857530594,), (0.34959060995550795,)]


In [23]:
collection_name = 'collection1'
new_text = 'Private and public blockchains serve different purposes, with public blockchains open to anyone and private blockchains restricted to authorized participants'
new_embedding = f"'{list(model.encode(new_text))}'"
query = "SELECT 1 - (embedding <=> {}) AS cosine_similarity FROM {};"
cur.execute(query.format(new_embedding, collection_name))
result = cur.fetchall()
print(result)

[(0.4027523696422577,), (0.023342888802289963,), (-0.05211000299781565,)]


## Delete Collection

In [24]:
cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_type = 'BASE TABLE';")
cur.fetchall()

[('dev_100',), ('dev_99',), ('collection1',)]

In [25]:
# drop a collection
collection_name = 'collection1'
cur.execute("DROP TABLE {};".format(collection_name))

In [26]:
cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_type = 'BASE TABLE';")
cur.fetchall()

[('dev_100',), ('dev_99',)]